In [2]:
from qutip import *
import numpy as np
import matplotlib.pyplot as plt
import cython

### Basis

In [3]:
atom_basis = {
    'b': 0,
    'g': 1,
    'u': 2,
    'x': 3
}
NUM_ATOM = len(atom_basis)

# polarisation states left with generic labels since we use the same compuational basis to represent multiple bases
pol_basis = {
    'p0': 0,
    'p1': 1,
    'p2': 2
}
NUM_POL = pol_basis


def def_ket(atom_state, pol_state):
    return tensor(basis(NUM_ATOM, atom_basis[atom_state]), basis(NUM_POL, pol_basis[pol_state]))


def def_bra(atom_state, pol_state):
    return def_ket(atom_state, pol_state).dag()


kets = {}
bras = {}

for a_state in atom_basis:
    for p_state in pol_basis:
        kets[f'{a_state}_{p_state}'] = def_ket(a_state, p_state)
        bras[f'{a_state}_{p_state}'] = def_bra(a_state, p_state)

ketbras = {}

for kkey in kets:
    for bkey in bras:
        ketbras[f'{kkey}_X_{bkey}'] = kets[kkey] * bras[bkey]

# identity functions
for state in atom_basis:
    ketbras[f'{state}_id'] = tensor(fock_dm(NUM_ATOM, atom_basis[state]), qeye(NUM_POL))


### Parameters

In [ ]:
STEPS_PER_SEC = 1000
NUM_COOPS = 25
NUM_TIMES = 25

gamma = 2*np.pi * 1
kappa = gamma * 1

gmin = np.sqrt(gamma/(2*kappa))
gmax = 10*np.sqrt(gamma/(2*kappa))
g_ratios = [np.sqrt(0.5**2*gamma/(2*kappa))] + np.logspace(np.log10(gmin), np.log10(gmax), NUM_COOPS)

t_min = -1
t_max = 2*np.log10(2)
times = np.logspace(t_min, t_max, NUM_TIMES)

### Efficiency Functions

In [ ]:
def find_efficiency(g_ratio, omega, delta, pulse_time, psi0, pulse_shape, added_time=1):

    time = pulse_time + added_time

    omega *= 2*np.pi
    delta *= 2*np.pi
    g0 = g_ratio*gamma

    H_args = {
        'omega': omega,
        'delta': delta,
        'g0': g0,
        'T': pulse_time,
        'wSTIRAP': np.pi/pulse_time
    }

    global current_time
    
    if time != current_time:
        rhs_clear()
        opts.rhs_reuse = False
        current_time = time

    # define time steps
    runtime = time/gamma
    num_steps = round(STEPS_PER_SEC*runtime + 1)
    if num_steps < 1001:
        num_steps = 1001
    
    t_res = 2
    int_time = round(runtime, t_res)*10**t_res
    t = np.linspace(0, int_time, num_steps)
    t /= 10**t_res

    H0 = -(ketbras[])
    H1 = -1/2*(ketbras['u_p0_X_x_p0'] + ketbras['x_p0_X_u_p0'])
    H2 = ketbras['g_id'] + ketbras['u_id']

    if pulse_shape == 'sin':
        pulse = 'np.piecewise(t, [t<T], [omega*np.sin(wSTIRAP*t)**2])'

    H=[[H0,'g0'], [H1, pulse], [H2, 'delta']]



    